# 自定义层构建
实现自定义层的最佳方法是扩展tf.keras.Layer类并实现：

- `__init__` ，可以在其中进行所有与输入无关的初始化
- build，这里可以知道输入张量的形状，并可以进行其余的初始化
- call，在这里进行前向传播

请注意，不一定必须到build被调用来创建变量时，也可以在中创建它们`__init__`。(如方法1、方法2)

但是，在build中创建它们的优点是，它可以根据层将在其上进行操作的输入的形状来进行后期变量创建 (方法3)

在`__init__`中创建变量将意味着需要明确指定创建变量所需的形状。


使用的主要数据结构是Layer。层封装了状态（层的“权重”）和从输入到输出的转换（“调用”，即层的前向传递）。

$y = x \cdot w +b $

假设$x \in R^{(2,2)}$, $w \in R^{(2,4)} $ , $b \in R^{(4)} $
返回 $y \in R^{(2,4)}$

In [5]:
!pip install tensorflow
from tensorflow.keras import layers
import tensorflow as tf

AttributeError: module 'numpy' has no attribute 'typeDict'

## 方法一

In [1]:
from tensorflow.keras import layers
import tensorflow as tf
#自定义全连接层
class Linear(layers.Layer):

    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__() #
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(initial_value=w_init(shape=(input_dim, units),
                                                  dtype='float32'),
                             trainable=True)
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(initial_value=b_init(shape=(units,),
                                                  dtype='float32'),
                             trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


x = tf.ones((2, 2))
linear_layer = Linear(4, 2)
y = linear_layer(x)
print(y)


tf.Tensor(
[[-0.05744977  0.00720382  0.02056642 -0.09241413]
 [-0.05744977  0.00720382  0.02056642 -0.09241413]], shape=(2, 4), dtype=float32)


In [2]:
linear_layer.trainable_variables

[<tf.Variable 'Variable:0' shape=(2, 4) dtype=float32, numpy=
 array([[-0.03379892,  0.03597696, -0.00595222, -0.03034527],
        [-0.02365085, -0.02877313,  0.02651864, -0.06206886]],
       dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]

In [54]:
linear_layer.w

<tf.Variable 'Variable:0' shape=(2, 4) dtype=float32, numpy=
array([[-0.00969894,  0.01236818, -0.07040638, -0.09068637],
       [ 0.05869705,  0.01278302, -0.05692462,  0.01281041]],
      dtype=float32)>

In [55]:
linear_layer.b

<tf.Variable 'Variable:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>

## 方法二
还可以使用更快的快捷方式为层添加权重：add_weight方法

In [4]:
class Linear(layers.Layer):

    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        self.w = self.add_weight(shape=(input_dim, units),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(shape=(units,),
                                 initializer='zeros',
                                 trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


x = tf.ones((2, 2))
linear_layer = Linear(4, 2)
y = linear_layer(x)
print(y)


tf.Tensor(
[[-0.03067889 -0.08481417 -0.06683034 -0.10394613]
 [-0.03067889 -0.08481417 -0.06683034 -0.10394613]], shape=(2, 4), dtype=float32)


## 方法三
在许多情况下，可能事先不知道输入的大小，并且想在实例化图层后的某个时间，在该值已知后懒惰地创建权重。

在Keras API中，建议build(inputs_shape)在图层的方法中创建图层权重。像这样：

In [5]:
class Linear(layers.Layer):

    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='random_normal',
                                 trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
    
    
x = tf.ones((2, 2))
linear_layer = Linear(units=4)
y = linear_layer(x)
print(y)

tf.Tensor(
[[ 0.03165464 -0.02968462  0.02153416  0.01931923]
 [ 0.03165464 -0.02968462  0.02153416  0.01931923]], shape=(2, 4), dtype=float32)


## 通过自定义层构建模型

In [7]:
inputs = tf.keras.Input(shape=(32,))  
x = Linear(units=64)(inputs) #tf.keras.layers.Dense()
x = tf.nn.relu(x) 
x = Linear(units=64)(x)
x = tf.nn.relu(x)
predictions = layers.Dense(10)(x)

In [8]:
model = tf.keras.Model(inputs=inputs, outputs=predictions)


model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

import numpy as np
data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))
model.fit(data, labels, batch_size=32, epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 1s 1ms/sample - loss: 13.1420 - accuracy: 0.1010
Epoch 2/5
1000/1000 [==============================] - 1s 690us/sample - loss: 26.3794 - accuracy: 0.1000
Epoch 3/5
1000/1000 [==============================] - 1s 508us/sample - loss: 50.1702 - accuracy: 0.0940
Epoch 4/5
1000/1000 [==============================] - 0s 484us/sample - loss: 80.5483 - accuracy: 0.0970
Epoch 5/5
1000/1000 [==============================] - 1s 834us/sample - loss: 114.2052 - accuracy: 0.0980
